In [2]:
! pip install -e .

Obtaining file:///
ERROR: file:/// does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [29]:
input="/app/data/outputs/2024.09.22/07.35.36_train_diffusion_unet_hybrid_pusht_image/checkpoints/latest.ckpt"
output="/app/data/outputs/2024.09.22/07.35.36_train_diffusion_unet_hybrid_pusht_image/checkpoints/output"
robot_ip="192.168.0.204"
match_dataset="/app/data/pusht_real/real_pusht_20230105"
match_episode=None
vis_camera_idx=0
init_joints=False
steps_per_inference=6
max_duration=60
frequency=10
command_latency=0.01

In [4]:
import time
from multiprocessing.managers import SharedMemoryManager
import click
import cv2
import numpy as np
import torch
import dill
import hydra
import pathlib
import skvideo.io
from omegaconf import OmegaConf
import scipy.spatial.transform as st
# from diffusion_policy.real_world.real_env import RealEnv
# from diffusion_policy.real_world.spacemouse_shared_memory import Spacemouse
from diffusion_policy.common.precise_sleep import precise_wait
from diffusion_policy.real_world.real_inference_util import (
    get_real_obs_resolution, 
    get_real_obs_dict)
from diffusion_policy.common.pytorch_util import dict_apply
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.base_image_policy import BaseImagePolicy
from diffusion_policy.common.cv2_util import get_image_transform

/root/miniconda3/envs/robodiff/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# load match_dataset
match_camera_idx = 0
episode_first_frame_map = dict()
if match_dataset is not None:
    match_dir = pathlib.Path(match_dataset)
    match_video_dir = match_dir.joinpath('videos')
    for vid_dir in match_video_dir.glob("*/"):
        episode_idx = int(vid_dir.stem)
        match_video_path = vid_dir.joinpath(f'{match_camera_idx}.mp4')
        if match_video_path.exists():
            frames = skvideo.io.vread(
                str(match_video_path), num_frames=1)
            episode_first_frame_map[episode_idx] = frames[0]
        break
print(f"Loaded initial frame for {len(episode_first_frame_map)} episodes")

Loaded initial frame for 1 episodes


In [6]:
# load checkpoint
ckpt_path = input
payload = torch.load(open(ckpt_path, 'rb'), pickle_module=dill)
cfg = payload['cfg']
cls = hydra.utils.get_class(cfg._target_)
workspace = cls(cfg)
workspace: BaseWorkspace
workspace.load_payload(payload, exclude_keys=None, include_keys=None)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['agent_pos']
using obs modality: rgb with keys: ['image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []


/root/miniconda3/envs/robodiff/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/robodiff/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Diffusion params: 6.502567e+07
Vision params: 1.119709e+07


In [7]:
# hacks for method-specific setup.
action_offset = 0
delta_action = False
if 'diffusion' in cfg.name:
    # diffusion model
    policy: BaseImagePolicy
    policy = workspace.model
    if cfg.training.use_ema:
        policy = workspace.ema_model
    device = torch.device('cuda')
    policy.eval().to(device)
    # set inference params
    policy.num_inference_steps = 16 # DDIM inference iterations
    policy.n_action_steps = policy.horizon - policy.n_obs_steps + 1
elif 'robomimic' in cfg.name:
    # BCRNN model
    policy: BaseImagePolicy
    policy = workspace.model
    device = torch.device('cuda')
    policy.eval().to(device)
    # BCRNN always has action horizon of 1
    steps_per_inference = 1
    action_offset = cfg.n_latency_steps
    delta_action = cfg.task.dataset.get('delta_action', False)
elif 'ibc' in cfg.name:
    policy: BaseImagePolicy
    policy = workspace.model
    policy.pred_n_iter = 5
    policy.pred_n_samples = 4096
    device = torch.device('cuda')
    policy.eval().to(device)
    steps_per_inference = 1
    action_offset = 1
    delta_action = cfg.task.dataset.get('delta_action', False)
else:
    raise RuntimeError("Unsupported policy type: ", cfg.name)

In [8]:
# setup experiment
dt = 1/frequency

obs_res = get_real_obs_resolution(cfg.task.shape_meta)
n_obs_steps = cfg.n_obs_steps
print("n_obs_steps: ", n_obs_steps)
print("steps_per_inference:", steps_per_inference)
print("action_offset:", action_offset)

n_obs_steps:  2
steps_per_inference: 6
action_offset: 0


In [33]:
from typing import Dict, Callable, Tuple
import numpy as np
from diffusion_policy.common.cv2_util import get_image_transform

def get_real_obs_dict(
        env_obs: Dict[str, np.ndarray], 
        shape_meta: dict,
        ) -> Dict[str, np.ndarray]:
    obs_dict_np = dict()
    obs_shape_meta = shape_meta['obs']
    for key, attr in obs_shape_meta.items():
        type = attr.get('type', 'low_dim')
        shape = attr.get('shape')
        if type == 'rgb':
            this_imgs_in = env_obs[key]
            t,hi,wi,ci = this_imgs_in.shape
            co,ho,wo = shape
            assert ci == co
            out_imgs = this_imgs_in
            if (ho != hi) or (wo != wi) or (this_imgs_in.dtype == np.uint8):
                tf = get_image_transform(
                    input_res=(wi,hi), 
                    output_res=(wo,ho), 
                    bgr_to_rgb=False)
                out_imgs = np.stack([tf(x) for x in this_imgs_in])
                if this_imgs_in.dtype == np.uint8:
                    out_imgs = out_imgs.astype(np.float32) / 255
            # THWC to TCHW
            obs_dict_np[key] = np.moveaxis(out_imgs,-1,1)
        elif type == 'low_dim':
            this_data_in = env_obs[key]
            if 'pose' in key and shape == (2,):
                # take X,Y coordinates
                this_data_in = this_data_in[...,[0,1]]
            obs_dict_np[key] = this_data_in
    return obs_dict_np


def get_real_obs_resolution(
        shape_meta: dict
        ) -> Tuple[int, int]:
    out_res = None
    obs_shape_meta = shape_meta['obs']
    for key, attr in obs_shape_meta.items():
        type = attr.get('type', 'low_dim')
        shape = attr.get('shape')
        if type == 'rgb':
            co,ho,wo = shape
            if out_res is None:
                out_res = (wo, ho)
            assert out_res == (wo, ho)
    return out_res

# define a fake env 
DEFAULT_OBS_KEY_MAP = {
    # robot
    'ActualTCPPose': 'robot_eef_pose',
    'ActualTCPSpeed': 'robot_eef_pose_vel',
    'ActualQ': 'robot_joint',
    'ActualQd': 'robot_joint_vel',
    # timestamps
    'step_idx': 'step_idx',
    'timestamp': 'timestamp'
}
class Env:
    def __init__(self,
            # required params
            output_dir,
            kuavo_robot_instance,
            
            # env params
            frequency=10,
            n_obs_steps=2,
            
            # obs
            obs_image_resolution=(640, 480),
            max_obs_buffer_size=30,
            camera_serial_number=None,
            obs_key_map=DEFAULT_OBS_KEY_MAP,
            obs_float32=False,
            
            # action
            max_pos_speed=0.25,
            max_rot_speed=0.6,
            # robot
            tcp_offset=0.13,
            init_joints=False,
            
            # video capture params
            video_capture_fps=30,
            video_capture_resolution=(1280,720),
            
            ### ???
            # saving params
            record_raw_video=True,
            thread_per_video=2,
            video_crf=21,
            # vis params
            enable_multi_cam_vis=True,
            multi_cam_vis_resolution=(1280,720),
            ### ???
            # shared memory 
            shm_manager=None
    ):
        print("Env init")  
        self.output_dir = output_dir
        self.kuavo_robot_instance = kuavo_robot_instance
        self.frequency = frequency
        self.n_obs_steps = n_obs_steps
        self.obs_image_resolution = obs_image_resolution
        self.max_obs_buffer_size = max_obs_buffer_size
        self.camera_serial_number = camera_serial_number
        self.obs_key_map = obs_key_map
        self.obs_float32 = obs_float32
        self.max_pos_speed = max_pos_speed
        self.max_rot_speed = max_rot_speed
        self.tcp_offset = tcp_offset
        self.init_joints = init_joints
        self.video_capture_fps = video_capture_fps
        self.video_capture_resolution = video_capture_resolution
        self.record_raw_video = record_raw_video
        self.thread_per_video = thread_per_video
        self.video_crf = video_crf
        self.enable_multi_cam_vis = enable_multi_cam_vis
        self.multi_cam_vis_resolution = multi_cam_vis_resolution
        self.shm_manager = shm_manager


    def get_fake_obs_dict(self):
        obs = dict()
        obs['anent_pos'] = np.random.rand(2)
        obs['img'] = np.random.rand(3,640,480)
        return obs

 

In [34]:
env:Env = Env(output, None, frequency=frequency, n_obs_steps=n_obs_steps, obs_image_resolution=obs_res)
env.get_fake_obs_dict()

Env init


{'anent_pos': array([0.37454012, 0.95071431]),
 'img': array([[[7.31993942e-01, 5.98658484e-01, 1.56018640e-01, ...,
          5.23097844e-01, 6.29398638e-01, 6.95748689e-01],
         [4.54541065e-01, 6.27558080e-01, 5.84314312e-01, ...,
          7.38033616e-01, 8.27986679e-02, 6.03152109e-01],
         [2.45349110e-01, 3.89295614e-01, 2.88693737e-01, ...,
          6.73432433e-01, 9.69912046e-01, 9.39007158e-02],
         ...,
         [5.58969363e-01, 5.76026859e-01, 9.01943320e-01, ...,
          7.40889624e-01, 7.82210445e-01, 7.57657125e-01],
         [4.31034703e-01, 7.47578759e-02, 2.99743171e-01, ...,
          7.43603380e-01, 2.35576933e-01, 9.83953269e-01],
         [2.12456817e-01, 1.93842413e-01, 5.66702916e-01, ...,
          9.09252102e-01, 7.72346211e-01, 6.08759915e-01]],
 
        [[2.06682862e-01, 7.90432806e-01, 1.97867048e-01, ...,
          3.72312954e-02, 3.23367946e-01, 2.65602402e-01],
         [4.53094478e-01, 6.30305867e-01, 4.29000558e-01, ...,
          6.

In [ ]:
# ========== policy control loop ==============
try:
    # start episode
    policy.reset()
    start_delay = 1.0
    eval_t_start = time.time() + start_delay
    t_start = time.monotonic() + start_delay
    env.start_episode(eval_t_start)
    # wait for 1/30 sec to get the closest frame actually
    # reduces overall latency
    frame_latency = 1/30
    precise_wait(eval_t_start - frame_latency, time_func=time.time)
    print("Started!")
    iter_idx = 0
    term_area_start_timestamp = float('inf')
    perv_target_pose = None
    while True:
        # calculate timing
        t_cycle_end = t_start + (iter_idx + steps_per_inference) * dt

        # get obs
        print('get_obs')
        obs = env.get_obs()
        obs_timestamps = obs['timestamp']
        print(f'Obs latency {time.time() - obs_timestamps[-1]}')

        # run inference
        with torch.no_grad():
            s = time.time()
            obs_dict_np = get_real_obs_dict(
                env_obs=obs, shape_meta=cfg.task.shape_meta)
            obs_dict = dict_apply(obs_dict_np, 
                lambda x: torch.from_numpy(x).unsqueeze(0).to(device))
            result = policy.predict_action(obs_dict)
            # this action starts from the first obs step
            action = result['action'][0].detach().to('cpu').numpy()
            print('Inference latency:', time.time() - s)
        
        # convert policy action to env actions
        if delta_action:
            assert len(action) == 1
            if perv_target_pose is None:
                perv_target_pose = obs['robot_eef_pose'][-1]
            this_target_pose = perv_target_pose.copy()
            this_target_pose[[0,1]] += action[-1]
            perv_target_pose = this_target_pose
            this_target_poses = np.expand_dims(this_target_pose, axis=0)
        else:
            this_target_poses = np.zeros((len(action), len(target_pose)), dtype=np.float64)
            this_target_poses[:] = target_pose
            this_target_poses[:,[0,1]] = action

        # deal with timing
        # the same step actions are always the target for
        action_timestamps = (np.arange(len(action), dtype=np.float64) + action_offset
            ) * dt + obs_timestamps[-1]
        action_exec_latency = 0.01
        curr_time = time.time()
        is_new = action_timestamps > (curr_time + action_exec_latency)
        if np.sum(is_new) == 0:
            # exceeded time budget, still do something
            this_target_poses = this_target_poses[[-1]]
            # schedule on next available step
            next_step_idx = int(np.ceil((curr_time - eval_t_start) / dt))
            action_timestamp = eval_t_start + (next_step_idx) * dt
            print('Over budget', action_timestamp - curr_time)
            action_timestamps = np.array([action_timestamp])
        else:
            this_target_poses = this_target_poses[is_new]
            action_timestamps = action_timestamps[is_new]

        # clip actions
        this_target_poses[:,:2] = np.clip(
            this_target_poses[:,:2], [0.25, -0.45], [0.77, 0.40])

        # execute actions
        env.exec_actions(
            actions=this_target_poses,
            timestamps=action_timestamps
        )
        print(f"Submitted {len(this_target_poses)} steps of actions.")

        # visualize
        episode_id = env.replay_buffer.n_episodes
        vis_img = obs[f'camera_{vis_camera_idx}'][-1]
        text = 'Episode: {}, Time: {:.1f}'.format(
            episode_id, time.monotonic() - t_start
        )
        cv2.putText(
            vis_img,
            text,
            (10,20),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.5,
            thickness=1,
            color=(255,255,255)
        )
        cv2.imshow('default', vis_img[...,::-1])


        key_stroke = cv2.pollKey()
        if key_stroke == ord('s'):
            # Stop episode
            # Hand control back to human
            env.end_episode()
            print('Stopped.')
            break

        # auto termination
        terminate = False
        if time.monotonic() - t_start > max_duration:
            terminate = True
            print('Terminated by the timeout!')

        term_pose = np.array([ 3.40948500e-01,  2.17721816e-01,  4.59076878e-02,  2.22014183e+00, -2.22184883e+00, -4.07186655e-04])
        curr_pose = obs['robot_eef_pose'][-1]
        dist = np.linalg.norm((curr_pose - term_pose)[:2], axis=-1)
        if dist < 0.03:
            # in termination area
            curr_timestamp = obs['timestamp'][-1]
            if term_area_start_timestamp > curr_timestamp:
                term_area_start_timestamp = curr_timestamp
            else:
                term_area_time = curr_timestamp - term_area_start_timestamp
                if term_area_time > 0.5:
                    terminate = True
                    print('Terminated by the policy!')
        else:
            # out of the area
            term_area_start_timestamp = float('inf')

        if terminate:
            env.end_episode()
            break

        # wait for execution
        precise_wait(t_cycle_end - frame_latency)
        iter_idx += steps_per_inference

except KeyboardInterrupt:
    print("Interrupted!")
    # stop robot.
    env.end_episode()

print("Stopped.")